In [229]:
from qiskit.optimization import QuadraticProgram
from qiskit.optimization.converters import InequalityToEquality, QuadraticProgramToQubo, LinearEqualityToPenalty
import numpy as np

n = 1 # personen
m = 2 # tasks

qubo = QuadraticProgram()

# define binary array of shape (n, m)
for k in range(n):
    for r in range(m):
        qubo.binary_var("x_"+str(k)+"_"+str(r))

# sample random costs per tasks
np.random.seed(seed=42)
c = np.random.rand(n, m)

### objective function
linear = {}

for k in range(n):
    for r in range(m):
        linear["x_"+str(k)+"_"+str(r)] = c[k, r]

qubo.minimize(linear=linear)

# constraint 1
for r in range(m):
    linear = {}
    for k in range(n):
        linear[f"x_{k}_{r}"] = 1
    qubo.linear_constraint(linear=linear, sense='EQ', rhs=1, name="constr_1_"+str(r))

# constraint 2
for k in range(n):
    linear = {}
    for r in range(m):
        linear[f"x_{k}_{r}"] = 1
    qubo.linear_constraint(linear=linear, sense='>=', rhs=1, name="constr_2_"+str(k))

print(qubo)

### convert inequality to equality constraints
ineq2eq = InequalityToEquality()
qubo_eq = ineq2eq.convert(qubo)

### add penalty term to constraints 
lineq2penalty = LinearEqualityToPenalty(penalty=50)
qubo_eq = lineq2penalty.convert(qubo_eq)

### convert integer variables to binary variables
qubo_eq = QuadraticProgramToQubo().convert(qubo_eq)
print(qubo_eq)

op, offset = qubo_eq.to_ising()
type(op)

print(f'Num of qubits {op.num_qubits}')


\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 0.374540118847 x_0_0 + 0.950714306410 x_0_1
Subject To
 constr_1_0: x_0_0 = 1
 constr_1_1: x_0_1 = 1
 constr_2_0: x_0_0 + x_0_1 >= 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1

Binaries
 x_0_0 x_0_1
End

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: - 199.625459881153 x_0_0 - 199.049285693590 x_0_1
      + 100 constr_2_0@int_slack@0 + [ 200 x_0_0^2 + 200 x_0_0*x_0_1
      - 200 x_0_0*constr_2_0@int_slack@0 + 200 x_0_1^2
      - 200 x_0_1*constr_2_0@int_slack@0 + 100 constr_2_0@int_slack@0^2 ]/2 +
      150
Subject To

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1
 0 <= constr_2_0@int_slack@0 <= 1

Binaries
 x_0_0 x_0_1 constr_2_0@int_slack@0
End

Num of qubits 3


### old qiskit aqua

In [230]:
from qiskit.aqua.algorithms import QAOA
from qiskit.aqua.components.optimizers import COBYLA
from qiskit.aqua.operators import Z, I

from qiskit import Aer
from qiskit.aqua import QuantumInstance as AquaQuantumInstance

backend = Aer.get_backend('qasm_simulator')

aqua_qinstance = AquaQuantumInstance(backend, seed_simulator=2, seed_transpiler=2)

qaoa = QAOA(optimizer=COBYLA(), quantum_instance=aqua_qinstance)
result = qaoa.compute_minimum_eigenvalue(op)

print(result.eigenvalue)
print(result.optimal_value)

result['eigenstate']

(-32.90381925068653+0j)
-32.90381925068653


{'011': 172,
 '111': 148,
 '100': 22,
 '110': 17,
 '001': 313,
 '010': 14,
 '000': 33,
 '101': 305}

In [231]:
highest_val = 0
highest_key = 0
for key, value in result['eigenstate'].items():
    if value > highest_val:
        highest_val = value
        highest_key = key

solution = [int(el) for el in highest_key]

In [232]:
solution

[0, 0, 1]

In [233]:

total_num_vars = len(solution)
num_slack = total_num_vars - n * m

solution_wo_slack = solution[:-num_slack]
print(solution_wo_slack)

qubo.get_feasibility_info(solution_wo_slack)

[0, 0]


(False,
 [],
  <qiskit.optimization.problems.linear_constraint.LinearConstraint at 0x7fcb7233f070>])

In [234]:
qubo

\ This file has been generated by DOcplex
\ ENCODING=ISO-8859-1
\Problem name: CPLEX

Minimize
 obj: 0.374540118847 x_0_0 + 0.950714306410 x_0_1
Subject To
 constr_1_0: x_0_0 = 1
 constr_1_1: x_0_1 = 1
 constr_2_0: x_0_0 + x_0_1 >= 1

Bounds
 0 <= x_0_0 <= 1
 0 <= x_0_1 <= 1

Binaries
 x_0_0 x_0_1
End

In [235]:
solution_reshaped = np.array(solution_wo_slack).reshape(n,m)

for person in range(n):
    for task in range(m):
        if solution_reshaped[person, task] == 1:
            print(f'Person {person} does task {task}')
        

### New Qiskit QAOA version

In [236]:
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.algorithms import QAOA, NumPyMinimumEigensolver
from qiskit.algorithms.optimizers import COBYLA, SPSA
from qiskit import  Aer
from qiskit.aqua import aqua_globals

backend = Aer.get_backend('qasm_simulator')

qinstance = QuantumInstance(backend, seed_simulator=2, seed_transpiler=2)

observable = Z ^ I
qaoa = QAOA(optimizer=COBYLA(), quantum_instance=qinstance)
result_test = qaoa.compute_minimum_eigenvalue(observable)
print(result_test.eigenvalue)

TypeError: Unsupported primitive type <class 'qiskit.aqua.operators.primitive_ops.pauli_op.PauliOp'> passed into StateFn factory constructor